In [11]:
type Result<'T> =
    | Success of 'T
    | Error of string

type LogState = { Logs: string list }

type ResultBuilder() =
    member _.Bind(x, f) = 
        match x with
        | Success v, (state: LogState) -> 
            let (result, newState) = f v
            match result with
            | Success _ ->
              (result, { state with Logs = state.Logs @ newState.Logs })
            | Error e -> (Error e, { state with Logs = state.Logs @ newState.Logs @ [sprintf "Error: %s" e] })
        | Error e, state ->
            (Error e, state)
            
    member _.Return(x) = 
        (Success x, { Logs = [] })
    
    member _.Zero() = 
        (Error "stopped", { Logs = ["Computation stopped due to zero."] })

    member _.Run x = x

let result = ResultBuilder()

let multiply10 x = 
    if x > 0 then (Success (x * 10), { Logs = [sprintf "multiply10 successful: %d" (x * 10)] })
    else (Error "x must be positive", { Logs = ["multiply10 failed: x must be positive"] })
    
let add1 x = 
    if x < 100 then (Success (x + 1), { Logs = [sprintf "add1 successful: %d" (x + 1)] })
    else (Error "result too large", { Logs = ["add1 failed: x too large"] })

let num1 = result {
    let! a = multiply10 5
    let! b = add1 a
    return b
}

printfn "num1: %A" num1

match num1 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs -> 
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs

let num2 = result {
    let! a = multiply10 50
    let! b = add1 a
    return b
}

match num2 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs


num1: (Success 51, { Logs = ["multiply10 successful: 50"; "add1 successful: 51"] })
Final result: 51
Logs: ["multiply10 successful: 50"; "add1 successful: 51"]
An error occurred: result too large
Logs: ["multiply10 successful: 500"; "add1 failed: x too large";
 "Error: result too large"]
